In [1]:
import plotly.express as px
from dash import Dash, html, dcc, callback, Input, Output
from Stage5Code import national_data, state_data
import pandas as pd

In [2]:
DEATHS = pd.read_csv('covid_deaths_usafacts.csv')
CASES = pd.read_csv('covid_confirmed_usafacts.csv')
POPULATION = pd.read_csv('covid_county_population_usafacts.csv')

In [3]:
case_totals_daily = national_data(CASES)
death_totals_daily = national_data(DEATHS)

states_case_totals_daily = pd.DataFrame()
states_death_totals_daily = pd.DataFrame()
STATES = ['NY', 'CA', 'TX', 'FL', 'IL', 'PA']
for state in STATES:
    states_case_totals_daily[state] = state_data(state, CASES).T
    states_death_totals_daily[state] = state_data(state, DEATHS).T

In [4]:
usa_cases = case_totals_daily.T.reset_index()
usa_cases.columns = ['Date', 'Cases']
usa_cases['Date'] = pd.to_datetime(usa_cases['Date'])

usa_deaths = death_totals_daily.T.reset_index()
usa_deaths.columns = ['Date', 'Deaths']
usa_deaths['Date'] = pd.to_datetime(usa_deaths['Date'])

states_cases = states_case_totals_daily.reset_index(names=['Date'])
states_cases['Date'] = pd.to_datetime(states_cases['Date'])

states_deaths = states_death_totals_daily.reset_index(names=['Date'])
states_deaths['Date'] = pd.to_datetime(states_deaths['Date'])

In [5]:
print('Constructing Dash app...')
app = Dash(__name__)
app.layout = (
    html.Div
        (
        [
            html.H1("Group 1 - Stage V", style={'text-align': 'center', 'color': '#333'}),
            html.H2("COVID-19 Cases and Deaths in the United States", style={'text-align': 'center', 'color': '#333'}),
            dcc.Graph(id='usa_graph'),
            html.Div(
                [
                    html.Label("Data Selection", style={'font-weight': 'bold'}),
                    dcc.Dropdown(
                        id='casedeath-dropdown1', clearable=False,
                        value='Cases', options=[
                            {'label': 'Cases', 'value': 'Cases'},
                            {'label': 'Deaths', 'value': 'Deaths'}
                        ]
                    )
                ], style={'margin-bottom': '20px', 'padding': '10px', 'border': '1px solid #ccc'}
            ),
            html.Div(
                [
                    html.Label("Scale Type", style={'font-weight': 'bold'}),
                    dcc.Dropdown(
                        id='scale-dropdown', clearable=False,
                        value='linear', options=[
                            {'label': 'Linear', 'value': 'linear'},
                            {'label': 'Logarithmic', 'value': 'log'}
                        ]
                    )
                ], style={'margin-bottom': '20px', 'padding': '10px', 'border': '1px solid #ccc'}
            ),
            html.Div(
                [
                    html.Label("Date Range", style={'font-weight': 'bold'}),
                    dcc.DatePickerRange(
                        id='date-picker-range',
                        start_date=usa_cases['Date'].min(),
                        end_date=usa_cases['Date'].max(),
                        display_format='YYYY-MM-DD'
                    )
                ], style={'margin-bottom': '20px', 'padding': '10px', 'border': '1px solid #ccc'}
            ),
            html.Div(
                [
                    html.Label("Trendline", style={'font-weight': 'bold'}),
                    dcc.RadioItems(
                        id='trendline-radio',
                        options=[
                            {'label': 'None', 'value': 'none'},
                            {'label': 'Linear', 'value': 'ols'}
                        ],
                        value='none'
                    )
                ], style={'margin-bottom': '20px', 'padding': '10px', 'border': '1px solid #ccc'}
            ),
        ], style={'padding': '20px', 'background-color': '#f9f9f9'}
    ),
    html.Div
        (
        [
            html.H2("7-Day Rolling Average for Cases/Deaths by State", style={'text-align': 'center', 'color': '#333'}),
            dcc.Graph(id='graph2'),
            html.Div(
                [
                    html.Label("Data Selection", style={'font-weight': 'bold'}),
                    dcc.Dropdown(
                        id='casedeath-dropdown2', clearable=False,
                        value='Cases', options=[
                            {'label': 'Cases', 'value': 'Cases'},
                            {'label': 'Deaths', 'value': 'Deaths'}
                        ]
                    )
                ], style={'margin-bottom': '20px', 'padding': '10px', 'border': '1px solid #ccc'}
            ),
            html.Div(
                [
                    html.Label("State", style={'font-weight': 'bold'}),
                    dcc.Checklist(
                        id='state-checklist',
                        options=[
                            {'label': 'New York', 'value': 'NY'},
                            {'label': 'California', 'value': 'CA'},
                            {'label': 'Texas', 'value': 'TX'},
                            {'label': 'Florida', 'value': 'FL'},
                            {'label': 'Illinois', 'value': 'IL'},
                            {'label': 'Pennsylvania', 'value': 'PA'}
                        ],
                        value=['NY'],  # Default selected values
                        labelStyle={'display': 'block'}
                    )
                ], style={'margin-bottom': '20px', 'padding': '10px', 'border': '1px solid #ccc'}
            )
        ]
    )
)


@app.callback(
    [
        Output('usa_graph', 'figure'),
        Output('date-picker-range', 'start_date'),
        Output('date-picker-range', 'end_date')
    ],
    [
        Input("casedeath-dropdown1", "value"),
        Input("scale-dropdown", "value"),
        Input("date-picker-range", "start_date"),
        Input("date-picker-range", "end_date"),
        Input("trendline-radio", "value")
    ]
)
def update_usa_figure(casedeath, scale, start_date, end_date, trendline):
    frame = usa_cases if casedeath == 'Cases' else usa_deaths
    color = 'blue' if casedeath == 'Cases' else 'red'

    mask = (frame['Date'] >= start_date) & (frame['Date'] <= end_date)
    filtered_frame = frame.loc[mask]

    fig = px.scatter(
        filtered_frame, x='Date', y=casedeath, render_mode="webgl",
        title=f'Total COVID-19 {casedeath} in the U.S.',
        trendline=None if trendline == 'none' else 'ols'
    ).update_traces(marker=dict(color=color))

    if scale == 'log':
        fig.update_layout(yaxis_type="log")

    return fig, start_date, end_date


@app.callback(
    [
        Output('graph2', 'figure'),
    ],
    [
        Input("casedeath-dropdown2", "value"),
        Input('state-checklist', 'value')
    ]
)
def update_state_figure(casedeath, states):
    frame = states_cases if casedeath == 'Cases' else states_deaths

    if not states:
        states = ['NY']  # Default to NY if no state is selected

    cmap = {
        "NY": "blue",
        "CA": "red",
        "TX": "green",
        "FL": "purple",
        "IL": "orange",
        "PA": "brown"}
    
    filtered_frame = frame[['Date'] + states].melt(id_vars=['Date'], var_name='State', value_name=casedeath)
    fig = px.scatter(filtered_frame, x='Date', y=casedeath, color='State', color_discrete_map=cmap,
                     title=f'COVID-19 {casedeath} by State',
                     render_mode="webgl", trendline='rolling', trendline_options=dict(window=7))
    fig.data = [t for t in fig.data if t.mode == "lines"]
    fig.update_traces(showlegend=True)

    return [fig]


app.run(jupyter_mode='tab', debug=False)

Constructing Dash app...
Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>